# Part I


Here we are downloading the results (though we're saving it here as SeriousStaff.csv and then using g_f1.csv).

## Table of Contents

   1. [Data Exploration](#section_1)
   2. [Word Processing](#section_2)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import numpy as np
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install -q kaggle
from google.colab import files

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nikitarozov","key":"9595eb57a7d29634a878701ac195d205"}'}

In [ ]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

In [ ]:
!kaggle datasets download -d gauthamp10/google-playstore-apps

 98% 202M/207M [00:01<00:00, 157MB/s]
100% 207M/207M [00:01<00:00, 142MB/s]


In [ ]:
!unzip google-playstore-apps.zip 

Archive:  google-playstore-apps.zip
  inflating: Google-Playstore.csv    


In [ ]:
data = pd.read_csv('Google-Playstore.csv')
data.head()

<div id='section_1'></div> 

# Data Exploration

Lets look at the data values in most important columns to check the dataset consistency


In [ ]:
print(data.Category.unique())
print(data.Rating.unique())
print(data["Rating Count"].unique())
print(data.Free.unique())
print(data.Price.unique())
print(data["Last Updated"].unique())
print(data['Content Rating'].unique())
print(data.dtypes)

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(12, 6), dpi=80)
data['Rating'].plot.box()

In [ ]:
figure(figsize=(20, 6), dpi=80)
data.sort_values('Minimum Installs')['Installs'].hist()

In [ ]:
data['Minimum Installs'].quantile(q=0.9)

100000.0

In [ ]:
education_data = data[data['Category'] == 'Education']
education_data.iloc[0]

In [ ]:
!pip install google-play-scraper
from google_play_scraper import app
result = app(education_data.iloc[0]['App Id'],
              lang='en',
              country='us' )
result['description']

In [ ]:
result1 = app("uz.yoqub.enuzaudio",
              lang='en',
              country='us' )
result1

In [ ]:
serious_dataset = data[(((data["Category"]=="Education") | (data["Category"]=="Educational")) & (data["Rating"]>=4.3) & (data["Maximum Installs"]>=50000))]
serious_dataset

In [ ]:
print(serious_dataset['App Name'].nunique()) # there are some duplication s olets keep last by sorting by installs
serious_dataset = serious_dataset.sort_values('Maximum Installs')
serious_dataset.head()

In [ ]:
serious_dataset = serious_dataset.drop_duplicates(subset=['App Name'], keep="last")

In [ ]:
print(serious_dataset.shape)

In [ ]:
def get_description(name_id):
  try:
    result = app(name_id,
                lang='en',
                country='us')
    return result['description']
  except:
    return "No description"


dataset['Description'] = serious_dataset['App Id'].apply(get_description)
serious_dataset.head()

In [ ]:
serious_dataset.to_csv('/content/drive/My Drive/EhealthData/serious_dataset.csv', index=False)

Which field of science the application should be related to

In [ ]:
data = pd.read_csv("/content/drive/My Drive/EhealthData/serious_dataset.csv")
data.head()

In [ ]:
def get_summary(name_id):
  try:
    result = app(name_id,
                lang='en',
                country='us')
    return result['summary']
  except:
    return "No summary"
data['Summary'] = data['App Id'].apply(get_summary)
data.to_csv('/content/drive/My Drive/EhealthData/serious_dataset.csv', index=False)

In [ ]:
data.head()

In [ ]:
my_part = data.iloc[200:230]
my_part['summary']= ''
my_part = my_part[['App Name','App Id', 'Description']]
my_part.to_csv("mypart.csv")

We looked in oxford dictionary to find the synonyms for the word learning

In [ ]:
from google.colab import files
files.upload()

In [ ]:
files.upload()

In [ ]:
with open('learning dictionary.txt') as f:
    learning_words = f.readlines()
with open('game dictionary.txt') as f:
    play_words = f.readlines()
print(learning_words)
print(play_words)

In [ ]:
for w in range(learning_words.__len__()):
  learning_words[w] = learning_words[w][0:-1]
learning_words = list(dict.fromkeys(learning_words))
learning_words.pop() #removing empty symbol
for w in range(play_words.__len__()):
  play_words[w] = play_words[w][0:-1]
play_words = list(dict.fromkeys(play_words))
print(play_words)
print(learning_words)

In [ ]:
def count_entrances(str1):
  counter = 0
  for d in learning_words:
    counter += str1.count(d)
  return counter

def count_entrances1(str1):
  counter = 0
  for d in play_words:
    counter += str1.count(d)
  return counter
data["Learning_frequency"] = data.Description.apply(count_entrances)
data["Gaming_frequency"] = data.Description.apply(count_entrances1)

In [ ]:
data.head() 

In [ ]:
print(data['Learning_frequency'].value_counts())
print(data['Gaming_frequency'].value_counts())

In [ ]:
data['Content Rating'].value_counts()

In [ ]:
candidat_data = data[((data['Content Rating']=='Everyone') & (data['Gaming_frequency']>=2) & (data['Learning_frequency']>=4))]
candidat_data.shape #1229, 1213

In [ ]:
candidat_data

Lets find application similar to duolinguo

In [ ]:
duo = candidat_data[candidat_data['App Name'].str.contains('Duolingo', regex=False)]
duo

In [ ]:
result = app(duo.iloc[0]['App Id'],
            lang='en',
            country='us')
result['similarApps']

In [ ]:
similar_data = candidat_data[candidat_data['App Id'].isin(result['similarApps'])]
similar_data.head()

In [ ]:
candidat_data.to_csv('/content/drive/My Drive/EhealthData/candidat_dataset.csv') #(Category=["Education","Educational"], Rating>=4.3, Gaming_frequency>=2,Learning_frequency>=4,Content Rating=Everyone))

In [ ]:
test_dataset = pd.read_csv("/content/drive/My Drive/EhealthData/Serious Staff.csv")
candidat_dataset = pd.read_csv("/content/drive/My Drive/EhealthData/candidat_dataset.csv")
test_set = set(test_dataset[(test_dataset['Serious']=='Yes') | (test_dataset['Serious']=='Y')|(test_dataset['Serious']=='yes')]['App Name'])
candidat_dataset = set(candidat_dataset['App Name'])
z = test_set.intersection(candidat_dataset)
set_recall = len(z) / len(test_set)
print(set_recall)

0.275


In [ ]:
g_1_f_1 = data[((data['Content Rating']=='Everyone') & (data['Gaming_frequency']>=1) & (data['Learning_frequency']>=1))]
#candidatg1_f1 = my_part data.iloc[200:230,160:190,130:160,300:450,500:550]
candidat_dataset = set(g_1_f_1['App Name'])
z = test_set.intersection(candidat_dataset)
set_recall = len(z) / len(test_set)
for (a)  
print(set_recall)
g_1_f_1.shape #1229, 1213

0.5


(2869, 28)

In [ ]:
g1_f1 = g_1_f_1.drop(columns=['Gaming_frequency','Learning_frequency','Summary'])
g1_f1.to_csv("/content/drive/My Drive/EhealthData/g1_f1.csv")

<div id='section_2'></div> 

# Words processing

Using lemmatizer for analyzing description

In [ ]:
!pip install ntlk
import nltk
nltk.download('punkt')
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

#sentence = "In this tutorial, we first define what we mean by area, and then we calculate the area under a curve by first approximating the area with rectangles and then making the width of the rectangles approach zero.\r\n \r\nIn the late 1600s, mathematicians discovered a much easier way to calculate the area under a curve, called The Fundamental Theorem of Calculus. This discovery started with the calculation of the slope of a tangent.\r\nIn the second part of this tutorial, we define the slope of the tangent to a curve, and we look at the connection between areas and tangents. \r\n\r\n\r\n* Aimed at students in their final two years of high school.\r\n\r\n* Studying mathematics is best done by working through examples and exercises. There are many interactive examples and exercises in this tutorial that must be completed to achieve 100% progress.\r\n\r\n* Written by a Maths teacher with 20 years of teaching experience.\r\n\r\n* Completely free (no Ads).\r\n\r\n* Works without internet so that you can learn while travelling in the train, bus etc.\r\n\r\n* Made with GameMaker (www.yoyogames.com)\r\n\r\n* Only a 14 MB download.\r\n\r\nAlso available on Google Play:\r\n * Calculus Tutorial 2: Limits\r\n * Calculus Tutorial 3: Continuous Functions\r\n * Calculus Tutorial 4: Derivative of a Function\r\n * Calculus Tutorial 4B: Velocity and Acceleration\r\n * Calculus Tutorial 5: Derivative of a Function II\r\n * Calculus Tutorial 6: Derivative of Trig Functions\r\n * Calculus Tutorial 7: Chain Rule Differentiation\r\n * Calculus Tutorial 8: Derivative of Inverse Functions\r\n * Calculus Tutorial 9: Implicit Differentiation"
sentence = "in in the in the"
punctuations="?:!.,;*()#$!@^&_-+="
sentence_words = nltk.word_tokenize(sentence)
for word in sentence_words:
    if word in punctuations:
        sentence_words.remove(word)

print(sentence_words)
print("{0:40}{1:40}".format("Word","Lemma"))
for word in sentence_words:
    print ("{0:20}{1:20}".format(word,wordnet_lemmatizer.lemmatize(word, pos="v")))

In [ ]:
data = pd.read_csv("/content/drive/My Drive/EhealthData/serious_dataset.csv")
data.head()

In [ ]:
from google.colab import files
files.upload()

In [ ]:
files.upload()

In [ ]:
with open('learning dictionary.txt') as f:
    learning_words = f.readlines()
with open('game dictionary.txt') as f:
    play_words = f.readlines()
print(learning_words)
print(play_words)

In [ ]:
for w in range(learning_words.__len__()):
  learning_words[w] = learning_words[w][0:-1]
learning_words = list(dict.fromkeys(learning_words))
learning_words.pop() #removing empty symbol
for w in range(play_words.__len__()):
  play_words[w] = play_words[w][0:-1]
play_words = list(dict.fromkeys(play_words))
print(play_words)
print(learning_words)

Making lemma text

In [ ]:
import re
r = re.compile(r'[^a-z ]')

def lemmat(sentence):
  sentence = sentence.replace('/',' ')
  sentence = sentence.replace('>',' ')
  sentence = sentence.lower()
  sentence = r.sub('', sentence)
  sentence_words = nltk.word_tokenize(sentence)
  
  new_sentence = list()
  for word in sentence_words:
    if(word.__len__() > 2):
      new_sentence.append(wordnet_lemmatizer.lemmatize(word, pos="v")) 
  return new_sentence

data['Lemmatize'] = data['Description'].apply(lemmat)
data.head()

In [ ]:
data['Description'].iloc[0]

In [ ]:
def count_entrances(str1):
  counter = 0
  for d in learning_words:
    counter += str1.count(d)
  return counter

def count_entrances1(str1):
  counter = 0
  for d in play_words:
    counter += str1.count(d)
  return counter
data["Learning_frequency"] = data.Lemmatize.apply(count_entrances)
data["Gaming_frequency"] = data.Lemmatize.apply(count_entrances1)
data.head()

In [ ]:
str1 = (data['Lemmatize'][data['App Name']=='Sunil Minglani'].to_string())
print(str1)

In [ ]:
candidat_data = data[((data['Content Rating']=='Everyone') & (data['Gaming_frequency']>=1) & (data['Learning_frequency']>=2))]
candidat_data.shape #329,445

In [ ]:
candidat_data.to_csv('/content/drive/My Drive/EhealthData/Lemma_dataset.csv') #(Category=["Education","Educational"], Rating>=4.3, Gaming_frequency>=2,Learning_frequency>=4,Content Rating=Everyone))

In [ ]:
test_dataset = pd.read_csv("/content/drive/My Drive/EhealthData/Serious Staff.csv")
candidat_dataset = pd.read_csv("/content/drive/My Drive/EhealthData/Lemma_dataset.csv")
test_set = set(test_dataset[test_dataset['Serious']=='Y']['App Name'])
candidat_dataset = set(candidat_dataset['App Name'])
z = test_set.intersection(candidat_dataset)
set_recall = len(z) / len(test_set)
print(set_recall)

0.0
